In [1]:
import copy
import torch
import torch.nn as nn
import numpy as np

from data_generation.generate_3d import ImageGenerator
from data_generation.generate_utils import get_batch
from data_generation.config import original_image_shape, cubic_simple_dims

from matplotlib import pyplot as plt
from models.net_utils import calculate_correctness_for_binary_input
from models.net_utils import get_best_device, prepare_image_for_network_input, prepare_image_for_analysis
from models.net_visualizations import three_d_visualize_model_progress, display3DImageMaskTuple

from models.cnnBinary import CNN_Binary
from data.data_utils import pad_image, divide_3d_image_into_patches, get_padded_patches

from sklearn.model_selection import train_test_split
from server_specific.server_utils import get_patients

In [2]:
patients = get_patients()
patients = np.array(patients)

train_patients = patients[0:700]
test_patients = patients[700:]

print("Train dataset:", train_patients.shape)
print("Test dataset:", test_patients.shape)


amt of detected_files:  1800
amt of patients:  800
Train dataset: (700,)
Test dataset: (100,)


In [3]:
# define which device is used for training
# todo replace with original image shape
default_image_shape = original_image_shape # only works for 3d

device = get_best_device()

if device == "mps":
    # mps is not supported for 3d
    device = "cpu"

torch.set_default_device(device)
print(f"Using {device} device. Every tensor created will be by default on {device}")

Using cuda device. Every tensor created will be by default on cuda


In [4]:
model = CNN_Binary(in_channels=1)
model.to(device);

In [5]:
print("----------------TRAINING-------------")

def coronary_arteries_in_masks(mask_batch):
    present = []
    for mask in mask_batch:
        coronary_arteries_present = torch.any(mask)

        present.append(coronary_arteries_present)
    
    present = torch.tensor(present, dtype=torch.float32)
    present = present[:, None]
    
    return present

----------------TRAINING-------------


In [10]:
padded_shape = (320, 512, 512)
patch_size = 64
block_shape = (patch_size, patch_size, patch_size)

def train_loop(model, loss_fn, optimizer, image, mask):
    model.train()
    
    avg_train_loss = 0

    image_patches, mask_patches = get_padded_patches(image, mask, patch_size)

    patch_shape = image_patches.shape
    amt_of_image_patches = patch_shape[0] * patch_shape[1] * patch_shape[2]
    # reconstructed_prediction_mask = np.zeros((padded_shape))
    
    correctness_matrix = np.array([0, 0, 0, 0])
    patch_counter = 0
    for i in range(patch_shape[0]):
        for j in range(patch_shape[1]):
            for k in range(patch_shape[2]):
                patch_counter += 1
                
                current_image_patch = image_patches[i, j, k]
                current_mask_patch = mask_patches[i, j, k]

                
                    
                current_image_patch = prepare_image_for_network_input(current_image_patch)
                current_mask_patch = prepare_image_for_network_input(current_mask_patch)

                optimizer.zero_grad()
                
                current_prediction = model(current_image_patch)
                ground_truth = coronary_arteries_in_masks(current_mask_patch)
                
                correctness_matrix += calculate_correctness_for_binary_input(current_prediction[-1].item(), ground_truth[-1].item())
                loss = loss_fn(current_prediction, ground_truth)

                loss.backward()
                optimizer.step()

                train_loss = loss.item()
                avg_train_loss += train_loss

                print(f"Patch number: {patch_counter} / {amt_of_image_patches}, Train loss: {train_loss:>8f}", end="\r")
    
    avg_train_loss /= amt_of_image_patches
    correctness_string = f"TP: {correctness_matrix[0]}, TN: {correctness_matrix[1]}, FP: {correctness_matrix[2]}, FN: {correctness_matrix[3]}"
                
    print(f"For this patient: average train loss: {avg_train_loss:>8f} | " + correctness_string + "\n")
    
    return avg_train_loss

In [11]:
def test_loop(model, loss_fn, image, mask):
    model.eval()

    test_loss = 0
    false_positives = 0
    false_negatives = 0
    correct = 0

    image_patches, mask_patches = get_padded_patches(image, mask, patch_size)

    patch_shape = image_patches.shape
    amt_of_image_patches = patch_shape[0] * patch_shape[1] * patch_shape[2]
    
    patch_counter = 0
    for i in range(patch_shape[0]):
        for j in range(patch_shape[1]):
            for k in range(patch_shape[2]):
                patch_counter += 1
                current_image_patch = image_patches[i, j, k]
                current_mask_patch = mask_patches[i, j, k]
                    
                current_image_patch = prepare_image_for_network_input(current_image_patch)
                current_mask_patch = prepare_image_for_network_input(current_mask_patch)

                with torch.no_grad():
                    current_prediction = model(current_image_patch)
                    ground_truth = coronary_arteries_in_masks(current_mask_patch)

                    patch_loss = loss_fn(current_prediction, ground_truth).item()
                    test_loss += patch_loss

    test_loss /= amt_of_image_patches

    print(f"Patch number: {patch_counter} / {amt_of_image_patches}, average test loss: {test_loss:>8f} | false positives: {false_positives}, false negatives: {false_negatives}\n")

    # returning for patience
    return test_loss

In [12]:
# resetting the model
model = CNN_Binary(in_channels=1)
model.to(device)

# running it
# param initialization for patience
best_loss = float('inf')  
best_model_weights = None  
patience_base_value = 8
patience = patience_base_value

loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
epochs = 2

In [13]:
try:
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        print("tuple of 7 train patients, 1 test patient")
        for i in range(100):
            print(f"processing tuple {i} from 100 tuples")
            
            current_train_patients = train_patients[i*7:(i+1)*7]
            current_test_patients = test_patients[i]
            
            average_train_losses = []
            average_test_losses = []
            
            for current_train_idx, current_train_patient in enumerate(current_train_patients):
                print(f"current train patient: {current_train_idx + 1} / 7")
                current_train_image, current_train_mask = current_train_patient.get_image_mask_tuple()
                current_average_train_loss = train_loop(model, loss_fn, optimizer, current_train_image, current_train_mask)
                average_train_losses.append(current_average_train_loss)
            # testing for each epoch to track the models performance during training.
            
            current_test_image, current_test_mask = current_train_patient.get_image_mask_tuple()
            test_loss = test_loop(model, loss_fn, current_test_image, current_test_mask)
            average_test_losses.append(test_loss)

            if test_loss < best_loss:
                best_loss = test_loss
                best_model_weights = copy.deepcopy(model.state_dict())
                patience = patience_base_value
            else:
                patience -= 1
                if patience <= 0:
                    break
            print("patience: ", patience)
    print("Done!")
except KeyboardInterrupt:
    print("training interrupted by the user")
    model.eval()

Epoch 1
-------------------------------
tuple of 7 train patients, 1 test patient
processing tuple 0 from 100 tuples
current train patient: 1 / 7
For this patient: average train loss: 0.391293 | TP: 19, TN: 250, FP: 22, FN: 29

current train patient: 2 / 7
For this patient: average train loss: 0.431439 | TP: 7, TN: 200, FP: 15, FN: 34

current train patient: 3 / 7
For this patient: average train loss: 0.387438 | TP: 17, TN: 242, FP: 22, FN: 39

current train patient: 4 / 7
For this patient: average train loss: 0.357701 | TP: 0, TN: 222, FP: 2, FN: 32

current train patient: 5 / 7
For this patient: average train loss: 0.358039 | TP: 1, TN: 277, FP: 4, FN: 38

current train patient: 6 / 7
For this patient: average train loss: 0.383979 | TP: 0, TN: 273, FP: 3, FN: 44

current train patient: 7 / 7
For this patient: average train loss: 0.345602 | TP: 0, TN: 223, FP: 2, FN: 31

Patch number: 256 / 256, average test loss: 0.374748 | false positives: 0, false negatives: 0

patience:  8
process

In [10]:
print("------INFERENCE--------")

'''
for i in range(10):
    mask, pred = default_model_progress_visualization_function(model, get_image_fct=image_generator.get_3DImage)
    print("jaccard score for above image: ", calculate_jaccard_score(mask, pred))
'''

------INFERENCE--------


'\nfor i in range(10):\n    mask, pred = default_model_progress_visualization_function(model, get_image_fct=image_generator.get_3DImage)\n    print("jaccard score for above image: ", calculate_jaccard_score(mask, pred))\n'

In [11]:
# torch.save(model.state_dict(), 'model_weights.pth')

for patient in test_patients:
    
